<a href="https://colab.research.google.com/github/ppfenninger/Sensorimotor_Learning_Final/blob/main/Sensorimotor_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Installation
!pip install pybullet > /dev/null 2>&1
!pip install git+https://github.com/taochenshh/easyrl.git > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install git+https://github.com/ppfenninger/airobot.git > /dev/null 2>&1
# !pip install git+https://github.com/ppfenninger/Sensorimotor_Learning_Final.git > /dev/null 2>&1
!pip uninstall torch -y
!pip install torch==1.12.0

Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Successfully uninstalled torch-2.0.0+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.


In [2]:
import os
import torch
import gym
import pprint
import time
import pybullet as p
import pybullet_data as pd
import pybullet_envs
import airobot as ar
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from typing import Any
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pylab
from dataclasses import dataclass
from gym import spaces
from gym.envs.registration import registry, register
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm.notebook import tqdm
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from copy import deepcopy
from itertools import count
from easyrl.agents.ppo_agent import PPOAgent
from easyrl.utils.common import save_traj
from easyrl.configs import cfg
from easyrl.configs import set_config
from easyrl.configs.basic_config import BasicConfig
from easyrl.configs.command_line import cfg_from_cmd
from easyrl.engine.ppo_engine import PPOEngine
from easyrl.models.categorical_policy import CategoricalPolicy
from easyrl.models.diag_gaussian_policy import DiagGaussianPolicy
from easyrl.models.mlp import MLP
from easyrl.models.value_net import ValueNet
from easyrl.agents.base_agent import BaseAgent
from easyrl.utils.torch_util import DictDataset
from easyrl.utils.torch_util import load_state_dict
from easyrl.utils.torch_util import load_torch_model
from easyrl.runner.nstep_runner import EpisodicRunner
from easyrl.utils.torch_util import save_model
from easyrl.utils.torch_util import action_entropy
from easyrl.utils.torch_util import action_from_dist
from easyrl.utils.torch_util import action_log_prob
from easyrl.utils.torch_util import clip_grad
from easyrl.utils.common import set_random_seed
from easyrl.utils.gym_util import make_vec_env
from easyrl.utils.common import load_from_json
from easyrl.utils.torch_util import freeze_model
from easyrl.utils.torch_util import move_to
from easyrl.utils.torch_util import torch_float
from easyrl.utils.torch_util import torch_to_np
from base64 import b64encode
from IPython import display as ipythondisplay

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/easyrl/envs/shmem_vec_env.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool: ctypes.c_bool}


In [32]:
del sys.modules["de_agent"]
del sys.modules["de_runner"]
del sys.modules["utils"]
del sys.modules["de_env"]
del sys.modules["de_engine"]

# install our library
!rm Sensorimotor_Learning_Final -r
!git clone -b testing https://github.com/ppfenninger/Sensorimotor_Learning_Final.git
import sys
sys.path.insert(0, './Sensorimotor_Learning_Final/deepexploration/')
import de_agent
from de_agent import DeepExplorationAgent
# import de_engine
import de_runner # this should work once everything compiles
from de_runner import DeepExplorationRunner
import utils
from utils import eval_agent, load_expert_agent, create_actor, create_critic

import de_env
from de_env import URRobotPusherGym

import de_engine
from de_engine import DeepExplorationEngine

module_name = __name__

env_name = 'URPusher-v1'
if env_name in registry.env_specs:
    del registry.env_specs[env_name]
register(
    id=env_name,
    entry_point=f'{module_name}:URRobotPusherGym',
)

Cloning into 'Sensorimotor_Learning_Final'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 274 (delta 71), reused 56 (delta 46), pack-reused 183
Receiving objects: 100% (274/274), 150.91 KiB | 16.77 MiB/s, done.
Resolving deltas: 100% (143/143), done.


# Setup

In [26]:
class DEConfig(BasicConfig):
    num_traj = 1

def set_configs(exp_name='de'):
    cfg.alg = DEConfig()
    cfg.alg.seed = 2957 #seed
    cfg.alg.num_envs = 1
    cfg.alg.episode_steps = 150
    cfg.alg.max_steps = 600000
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = 'URPusher-v1'
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += f'/{exp_name}'
    cfg.alg.policy_lr: float = 3e-4
    cfg.alg.value_lr: float = 1e-3
    cfg.alg.linear_decay_lr: bool = False
    cfg.alg.max_decay_steps: int = 1e6
    cfg.alg.num_envs: int = 8
    cfg.alg.eval_num_envs: int = None
    cfg.alg.opt_epochs: int = 10
    cfg.alg.normalize_adv: bool = True
    cfg.alg.clip_vf_loss: bool = False
    cfg.alg.vf_loss_type: str = 'mse'
    cfg.alg.vf_coef: float = 0.5
    cfg.alg.ent_coef: float = 0.01
    cfg.alg.clip_range: float = 0.2
    cfg.alg.linear_decay_clip_range: bool = False
    cfg.alg.gae_lambda: float = 0.95
    cfg.alg.rew_discount: float = 0.99
    cfg.alg.use_amsgrad: bool = True
    cfg.alg.sgd: bool = False
    cfg.alg.momentum: float = 0.00
    cfg.alg.tanh_on_dist: bool = False
    cfg.alg.std_cond_in: bool = False
    cfg.alg.log_interval = 5
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'====================================')

In [6]:
# Runner Tests
set_configs()
env = URRobotPusherGym(max_episode_length=30)

critics = []
for index in range(4):
  ob_size = env.observation_space.shape[0]
  critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
  critic = ValueNet(critic_body, in_features=64)
  critics.append(critic)

actor = create_actor(env=env)
agent = DeepExplorationAgent(actor=actor, critics=critics, env=env)
runner = DeepExplorationRunner(agent=agent, env=env)

# traj = runner(time_steps=cfg.alg.episode_steps)

      Device:cuda


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


i have this many people telling me what to do 4


In [27]:
def train_de(max_steps=200000):
    set_configs()
    cfg.alg.num_envs = 1
    cfg.alg.episode_steps = 100
    cfg.alg.max_steps = max_steps
    cfg.alg.deque_size = 20
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = 'URPusher-v1'
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += '/'
    cfg.alg.save_dir += 'push'
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'      Total number of steps:{cfg.alg.max_steps}')
    print(f'====================================')

    set_random_seed(cfg.alg.seed)
    env_kwargs=dict()
    env = make_vec_env(cfg.alg.env_name,
                       cfg.alg.num_envs,
                       seed=cfg.alg.seed,
                       env_kwargs=env_kwargs)
    env.reset()
    ob_size = env.observation_space.shape[0]

    actor_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)

    critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
  
    if isinstance(env.action_space, gym.spaces.Discrete):
        act_size = env.action_space.n
        actor = CategoricalPolicy(actor_body,
                                 in_features=64,
                                 action_dim=act_size)
        
    elif isinstance(env.action_space, gym.spaces.Box):
        act_size = env.action_space.shape[0]
        actor = DiagGaussianPolicy(actor_body,
                                   in_features=64,
                                   action_dim=act_size,
                                   tanh_on_dist=cfg.alg.tanh_on_dist,
                                   std_cond_in=cfg.alg.std_cond_in)
    else:
        raise TypeError(f'Unknown action space type: {env.action_space}')

    critics = [] # ValueNet(critic_body, in_features=64) # TODO: get critics

    for critic in range(2):
      critics.append(create_critic(env))


    agent = DeepExplorationAgent(actor=actor, critics=critics, env=env)
    runner = DeepExplorationRunner(agent=agent, env=env)
    engine = DeepExplorationEngine(agent=agent, runner=runner, env=env)
    engine.train()
    stat_info, raw_traj_info = engine.eval(render=False, save_eval_traj=True, eval_num=1, sleep_time=0.0)
    pprint.pprint(stat_info)
    return cfg.alg.save_dir

# Testing

In [ ]:
saved_dir_pusher = train_de(max_steps = 100000)

[INFO][2023-05-07 17:11:22]: Creating 1 environments.
INFO:EasyRL:Creating 1 environments.


      Device:cuda
      Device:cuda
      Total number of steps:100000


[ERROR][2023-05-07 17:11:23]: Not a valid git repo: /usr/local/lib/python3.10/dist-packages
ERROR:EasyRL:Not a valid git repo: /usr/local/lib/python3.10/dist-packages


i have this many people telling me what to do 2
iter: 0
We are done! 29


[INFO][2023-05-07 17:11:23]: Exploration steps: 0
INFO:EasyRL:Exploration steps: 0
[INFO][2023-05-07 17:11:23]: Saving checkpoint: /content/data/push/seed_2957/model/ckpt_000000000000.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2957/model/ckpt_000000000000.pt.
[INFO][2023-05-07 17:11:23]: Saving checkpoint: /content/data/push/seed_2957/model/model_best.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2957/model/model_best.pt.


...

We are done! 29
iter: 101
iter: 102
iter: 103
iter: 104
iter: 105
iter: 106
iter: 107
iter: 108
iter: 109
iter: 110
iter: 111
iter: 112
iter: 113
iter: 114
iter: 115
iter: 116
iter: 117
iter: 118
iter: 119
iter: 120
iter: 121
iter: 122
iter: 123
iter: 124
iter: 125
iter: 126
iter: 127
iter: 128
iter: 129
iter: 130
iter: 131
iter: 132
iter: 133
iter: 134
iter: 135
iter: 136
iter: 137
iter: 138
iter: 139
iter: 140
iter: 141
iter: 142
iter: 143
iter: 144
iter: 145
iter: 146
iter: 147
iter: 148
iter: 149
iter: 150
iter: 151
iter: 152
iter: 153
iter: 154
iter: 155
iter: 156
iter: 157
iter: 158
iter: 159
iter: 160
iter: 161
iter: 162
iter: 163
iter: 164
iter: 165
iter: 166
iter: 167
iter: 168
iter: 169
iter: 170
iter: 171
iter: 172
iter: 173
iter: 174
iter: 175
iter: 176
iter: 177
iter: 178
iter: 179
iter: 180
iter: 181
iter: 182
iter: 183
iter: 184
iter: 185
iter: 186
iter: 187
iter: 188
iter: 189
iter: 190
iter: 191
iter: 192
iter: 193
iter: 194
iter: 195
iter: 196
iter: 197
iter: 198
iter

[INFO][2023-05-07 17:17:26]: Exploration steps: 20000
INFO:EasyRL:Exploration steps: 20000
[INFO][2023-05-07 17:17:26]: Saving checkpoint: /content/data/push/seed_2957/model/ckpt_000000020000.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2957/model/ckpt_000000020000.pt.
[INFO][2023-05-07 17:17:26]: Saving checkpoint: /content/data/push/seed_2957/model/model_best.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2957/model/model_best.pt.


iter: 200
We are done! 29
We are done! 29
iter: 201
iter: 202
iter: 203
iter: 204
iter: 205
iter: 206
iter: 207
iter: 208
iter: 209
iter: 210
iter: 211
iter: 212
iter: 213
iter: 214
iter: 215
iter: 216
iter: 217
iter: 218
iter: 219
iter: 220
iter: 221
iter: 222
iter: 223
iter: 224
iter: 225
iter: 226
iter: 227
iter: 228
iter: 229
iter: 230
iter: 231
iter: 232
iter: 233
iter: 234
iter: 235
iter: 236
iter: 237
iter: 238
iter: 239
iter: 240
iter: 241
iter: 242
iter: 243
iter: 244
iter: 245
iter: 246
iter: 247
iter: 248
iter: 249
iter: 250
iter: 251
iter: 252
iter: 253
iter: 254
iter: 255
iter: 256
iter: 257
iter: 258
iter: 259
iter: 260
iter: 261
iter: 262
iter: 263
iter: 264
iter: 265
iter: 266
iter: 267
iter: 268
iter: 269
iter: 270
iter: 271
iter: 272
iter: 273
iter: 274
iter: 275
iter: 276
iter: 277
iter: 278
iter: 279
iter: 280
iter: 281
iter: 282
iter: 283
iter: 284
iter: 285
iter: 286
iter: 287
iter: 288
iter: 289
iter: 290
iter: 291
iter: 292
iter: 293
iter: 294
iter: 295
iter: 29

[INFO][2023-05-07 17:20:22]: Exploration steps: 30000
INFO:EasyRL:Exploration steps: 30000
[INFO][2023-05-07 17:20:22]: Saving checkpoint: /content/data/push/seed_2957/model/ckpt_000000030000.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2957/model/ckpt_000000030000.pt.
[INFO][2023-05-07 17:20:22]: Saving checkpoint: /content/data/push/seed_2957/model/model_best.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2957/model/model_best.pt.


iter: 300
We are done! 29
We are done! 29
iter: 301
iter: 302
iter: 303
iter: 304
iter: 305
iter: 306
iter: 307
iter: 308
iter: 309
iter: 310
iter: 311
iter: 312
iter: 313
iter: 314
iter: 315
iter: 316
iter: 317
iter: 318
iter: 319
iter: 320
iter: 321
iter: 322
iter: 323
iter: 324
iter: 325
iter: 326
iter: 327
iter: 328
iter: 329
iter: 330
iter: 331
iter: 332
iter: 333
iter: 334
iter: 335
iter: 336
iter: 337
iter: 338
iter: 339
iter: 340
iter: 341
iter: 342
iter: 343
iter: 344
iter: 345
iter: 346
iter: 347
iter: 348
iter: 349
iter: 350
iter: 351
iter: 352
iter: 353
iter: 354
iter: 355
iter: 356
iter: 357
iter: 358
iter: 359
iter: 360
iter: 361
iter: 362
iter: 363
iter: 364
iter: 365
iter: 366
iter: 367
iter: 368
iter: 369
iter: 370
iter: 371
iter: 372
iter: 373
iter: 374
iter: 375
iter: 376
iter: 377
iter: 378
iter: 379
iter: 380
iter: 381
iter: 382


In [ ]:
print(saved_dir_pusher)

def read_tf_log(log_dir):
    log_dir = Path(log_dir)
    log_files = list(log_dir.glob(f'**/events.*'))
    print(log_files)
    if len(log_files) < 1:
        return None
    log_file = log_files[0]
    event_acc = EventAccumulator(log_file.as_posix())
    event_acc.Reload()
    tags = event_acc.Tags()
    try:
        scalar_success = event_acc.Scalars('train/episode_success')
        success_rate = [x.value for x in scalar_success]
        steps = [x.step for x in scalar_success]
        scalar_return = event_acc.Scalars('train/episode_return/mean')
        returns = [x.value for x in scalar_return]
    except:
        return None
    return steps, returns, success_rate

steps, returns, success_rates = read_tf_log(saved_dir_pusher)

In [ ]:
plt.plot(steps, returns)
print(returns)
print(steps)
print(success_rates)